In [ ]:
from google.colab import drive
drive.mount("/content/drive")
HOME = 'drive/MyDrive'
!pip install torchtext==0.4

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import random
import pandas as pd
import numpy as np

In [ ]:
dataset_quotes = []

import os
with open("drive/MyDrive/Data/PlainSimpleGarak-data.txt", 'r') as f: # open in readonly mode
  while True:
    line = f.readline().strip()
    if line == '':
        break
    else:
      dataset_quotes.append(line)

dataset_startwords = []


for s in dataset_quotes:
  dataset_startwords.append(s.split()[0].lower())

dataset_raw = pd.read_csv('drive/MyDrive/Data/quotes.csv', error_bad_lines=False, engine='python')

dataset_quotes_nonascii = list(dataset_raw["Quote"] )

for s in dataset_quotes_nonascii:
  if (len(s) == len(s.encode())):
    if ("$" not in s and "&" not in s):
      dataset_quotes.append(s)

In [ ]:
def preprocess_txt(dataset_quotes, appende=True):
  for i in range(len(dataset_quotes)):
    dataset_quotes[i] = dataset_quotes[i].replace('\t', ' ')
    dataset_quotes[i] = dataset_quotes[i].replace('--', ' ')
    dataset_quotes[i] = dataset_quotes[i].replace(' - ', ' ')
    dataset_quotes[i] = dataset_quotes[i].replace('%', ' percent')
    dataset_quotes[i] = dataset_quotes[i].replace('[', '')
    dataset_quotes[i] = dataset_quotes[i].replace(']', '')
    dataset_quotes[i] = dataset_quotes[i].replace(':', '')
    dataset_quotes[i] = dataset_quotes[i].replace(';', '')
    dataset_quotes[i] = dataset_quotes[i].replace('?', '')
    dataset_quotes[i] = dataset_quotes[i].replace('#', '')
    dataset_quotes[i] = dataset_quotes[i].replace('!', '')
    dataset_quotes[i] = dataset_quotes[i].replace('/', '')
    dataset_quotes[i] = dataset_quotes[i].replace('.', '')
    dataset_quotes[i] = dataset_quotes[i].replace(',', '')
    dataset_quotes[i] = dataset_quotes[i].replace('(', '')
    dataset_quotes[i] = dataset_quotes[i].replace(')', '')
    dataset_quotes[i] = dataset_quotes[i].replace('*', '')
    dataset_quotes[i] = dataset_quotes[i].replace('+', '')
    dataset_quotes[i] = dataset_quotes[i].replace('\"', '')
    #dataset_quotes[i] = dataset_quotes[i].replace('\'', '')
    dataset_quotes[i] = dataset_quotes[i].lower()
    if (appende == True):
      dataset_quotes[i] = dataset_quotes[i] + " END"

preprocess_txt(dataset_quotes)
preprocess_txt(dataset_startwords, False)

dataset_startwords = list(set(dataset_startwords))

random.shuffle(dataset_quotes)
random.shuffle(dataset_startwords)

In [ ]:
txt_quotes = ""
for s in dataset_quotes:
  txt_quotes = txt_quotes + " " + s
txt_quotes = txt_quotes.split()

list_words = list(set(txt_quotes))
word2index = {tkn: i for i, tkn in enumerate(list_words, 1)}
word2index['<UNKNOWN>']=0
index2word = {v: k for k, v in word2index.items()}

In [ ]:
def string_to_trainpair(string, word2index):
  string = string.split()
  encoded = [word2index[wd] for wd in string]
  input_str, target_str = encoded[:-1], encoded[1:]
  input_str = torch.LongTensor(input_str).unsqueeze(0)
  target_str = torch.LongTensor(target_str).unsqueeze(0)
  return input_str, target_str

def encode(string, word2index):
  return torch.LongTensor([[word2index[token] for token in string.split()]])

def decode(vec, index2word):
  return [index2word.get(x) for x in vec]

In [ ]:
dataset_train = []
dataset_target = []

for s in dataset_quotes:
  input, target = string_to_trainpair(s, word2index)
  dataset_train.append(input)
  dataset_target.append(target)

In [ ]:
class Net(nn.Module):
  def __init__(self, embed_size, input_dim, hidden_dim, batch_first=True):
    super(Net, self).__init__()

    self.n_layers = 1 #unused
    self.hidden_dim = hidden_dim

    self.embedding_layer = nn.Embedding(num_embeddings=embed_size, embedding_dim=input_dim)
    #1
    self.rnn_layer1 = nn.GRU(input_dim, hidden_dim, batch_first=batch_first)
    self.linear1 = nn.Linear(hidden_dim, embed_size)
    #2
    self.rnn_layer2 = nn.GRU(input_dim, hidden_dim, batch_first=batch_first)
    self.linear2 = nn.Linear(hidden_dim, embed_size)

  def forward(self, x):
    output = self.embedding_layer(x)

    if self.training:
      output_1, hidden_1 = self.rnn_layer1(output)
      output_1 = self.linear1(output_1)
      output_2, hidden_2 = self.rnn_layer2(output)
      output_2 = self.linear2(output_2)

      output = 0.5 * (output_1 + output_2)

    else:
      if (random.randrange(2) == 0):
        output, hidden = self.rnn_layer1(output)
        output = self.linear1(output)
      else:
        output, hidden = self.rnn_layer2(output)
        output = self.linear2(output)
    return output.view(-1, output.size(2))

  def init_hidden(self, batch_size):
      return torch.zeros(self.n_layers, batch_size, self.hidden_dim)  #unused

In [ ]:
vocab_size = len(word2index)
input_size =  256
hidden_size = 512

In [ ]:
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters())

In [ ]:
def test_model(model, word2index, index2word, string=""):
  model.eval()

  randword = dataset_startwords[random.randrange(0,len(dataset_startwords))]

  for rep in range(2):
    eval_input = encode(randword, word2index).cuda()
    print("INPUT: " + string + " " + randword)

    for i in range(50):
      
      output = model(eval_input)
      pred = output.softmax(-1).argmax(-1)

      eval_input = torch.cat((eval_input,pred[-1].unsqueeze(0).unsqueeze(0)),1)
      
      if (word2index['END'] in eval_input[0]):
        break

    print(" ".join(decode(eval_input.tolist()[0],index2word)))

In [ ]:
for epoch in range(1001):
  model.train()

  for input, target in zip(dataset_train, dataset_target):
    optimizer.zero_grad()
    output = model(input.cuda())
    loss = criterion(output, target.view(-1).cuda())
    loss.backward()
    optimizer.step()
    break

  print("[{:02d}/1001] {:.4f} ".format(epoch+1, loss))
  pred = output.softmax(-1).argmax(-1)
  print(" ".join([index2word[int(input[0][0])]] + decode(pred.tolist(),index2word)))

  if (epoch % 5 == 0):
    #torch.save(model.state_dict(), "drive/MyDrive/Data/" + "Checkpoint-" + str(epoch) )
    torch.save(
        {'model_state_dict': model.state_dict(),
         'optimizer_state_dict': optimizer.state_dict(),},
        'drive/MyDrive/Data/Checkpoint/' + "CPOINT-" + str(epoch)
               )

  print("===========================================================================")

  test_model(model, word2index, index2word)
  test_model(model, word2index, index2word, "i am plain simple garak")

  print("===========================================================================")

  print()